In [1]:
try: use_MASTER
except: use_MASTER = False

if not use_MASTER:
    j='b2345a'#all samples
    # j='b2345_prescaled'
    args =  {
            'j' : j,
            'feautures_ind':4,
            'data_folder' : 'Data_',
            # 'data_folder' : 'age_prescaled',
            
            # create visualization (umap, dbscan are always shown)
            'visualize':True,'plotUMAP' : True,
            # show figures in notebook
            'show' : True, 'saveSVG' :False,
            'recalculate_umap' : False,# calculate umap
            'recalculate_db' : True,# calculate db
            'create_adjusted' : False,
            'compare' : False,# compare 11,stroma and rest of the cells     
            
            }


In [2]:



import json

import sys,pathlib, os, warnings
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
# from tqdm import tqdm_notebook, tqdm
import datetime,time
# start = time.process_time()

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
# from typing import List

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
# from IPython.display import Image, display
# from matplotlib.pyplot import rc_context
# from epinuc import *
sys.path.append(f'{str(pathlib.Path.home())}/Desktop/breast_cancer_PHD_research/functions/')

from parent_class import Parent 
# import  print_n_log 



# from usefull_functions_new import *
from usefull_functions_new import cluster_samples_probability,removeFeatures

# from functions import *
# # from plot_functions import *
# # from clustering_functions import *
from xg_shap import * 
from plot_functions_new import *




plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180)  # according to screen width

%load_ext autoreload
%autoreload 2

In [3]:
from pandas import DataFrame,Series
# from typing import List
from numpy import ndarray
from matplotlib.pyplot import figure

In [4]:


class Config(Parent):

    def __init__(self, **args):
        '''
        args is a dictionary of parameters inserted by user
        '''
        loaded_args = self.args_from_json() 
        args = args if not loaded_args else loaded_args
        self.__dict__.update(args)
        
        self.isSingleSample = True if 'b' not in self.j else False
        self.isCellCycle = True if '.2'  in self.j else False#18.2,14.2
        
        self.home = str(pathlib.Path.home())
        self.dir_functions = f'{self.home}/Desktop/breast_cancer_PHD_research/functions/'
        # self.dir_data_parent = f'{self.home}/Dropbox/CyTOF_Breast/data_yishai/'
        self.dir_data_parent = f'{self.home}/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/'
        
        self.dir_data = f'{self.dir_data_parent}{self.data_folder}/'
        print(self.dir_data )
        self.dir_index =  f'{self.dir_data_parent}indexes/'
        # self.dir_plots = f'{self.home}/Desktop/plots/_sample_{self.j}_{datetime.date.today().strftime("%d%m%Y")}_{self.data_folder}/'
        self.dir_plots = f'{self.dir_data_parent}plots/_sample_{self.j}_{datetime.date.today().strftime("%d%m%Y")}_{self.data_folder}/'
        #data for tetsing
        self.save_add =  f'{self.dir_data_parent}'
        
        # self.params = ['iden', 'epi', 'all','cell'][self.feautures_ind]#params (for db and umap)
        
        # self.print()
        
        self.folderExists(self.dir_plots)

    def update(self, **kwargs):
        
        self.__dict__.update(kwargs)
    def args_from_json(self):
        # if thers an external program running script (saving a json file) - take it
        # otherwise take the input j
        try:   
            with open('args.json', 'r') as f: 
                args =  json.load(f)#load args dictionary from file
            os.remove('args.json')
            print('loaded from file')
        except:
            args = None
        return args
        
    def getNames(self,names):
        self.name = list(names)[self.feautures_ind]

        self.title = f'samp{self.j} {self.name} Based '
        self.figname = self.title.replace(' ','_')      
    def print(self):
        print(f"Sample is: {'single' if self.isSingleSample else 'batch'}")
        # print(f'title; {title}, figname; {figname}, params (for db and umap);{params}')
        print(f'title; {self.title}, figname; {self.figname}')#, params (for db and umap);{self.params}

    def get_dict(self):
        """
        Prints all attribute of the config object
        """
       
        config_dict = {}
        for i in (vars(self)):
            #print("{0:10}: {1}".format(i, vars(self)[i]))
            config_dict[i] = vars(self)[i]
        
        return config_dict


config = Config(**args)



/Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/Data_/
The directory already exists!


In [5]:
try: # json instead of pickle - slower but readable
    with open(f'{config.dir_data}{config.j}_sample.json', "r") as f:
        df = pd.DataFrame.from_dict(json.load(f)).reset_index(drop=True)
    with open(f'{config.dir_data}{config.j}_features.json', "r") as f:
        names_dict = json.load(f)
    print (f'sample {config.j} loaded from file')
        
        
except:# json file doesnt exist - create it
    df1 = config.pickle_load('k'+config.j+'_dict', config.dir_data)['k'].reset_index(drop=True)
    names_dict1 = config.pickle_load('k'+config.j+'_names_dict', config.dir_data)
    with open(f'{config.dir_data}{config.j}_sample.json', "w") as f:
        json.dump(df1.to_dict(),f, )#indent = 4
    with open(f'{config.dir_data}{config.j}_features.json', "w") as f:
        json.dump(names_dict1,f, )#indent = 4 
        
    with open(f'{config.dir_data}{config.j}_sample.json', "r") as f:
        df = pd.DataFrame.from_dict(json.load(f)).reset_index(drop=True)
    with open(f'{config.dir_data}{config.j}_features.json', "r") as f:
        names_dict = json.load(f) 
    print((df1==df).all().all())
    print(names_dict1==names_dict) 
    print('json files created and loaded')  
del f

sample b2345a loaded from file


In [6]:
config.dir_data

'/Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/Data_/'

In [7]:
config.get_attribute()#print all attributes (config info)      

j         : b2345a
feautures_ind: 4
data_folder: Data_
visualize : True
plotUMAP  : True
show      : True
saveSVG   : False
recalculate_umap: False
recalculate_db: True
create_adjusted: False
compare   : False
isSingleSample: False
isCellCycle: False
home      : /Users/yishai
dir_functions: /Users/yishai/Desktop/breast_cancer_PHD_research/functions/
dir_data_parent: /Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/
dir_data  : /Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/Data_/
dir_index : /Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/indexes/
dir_plots : /Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/plots/_sample_b2345a_05062024_Data_/
save_add  : /Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/


In [8]:
if not use_MASTER:
    LEN = 1000
    
    ind = list(df.index)
    np.random.shuffle(ind)  
    df = df.loc[ind[:LEN]].reset_index(drop=True)
    del ind
    print (f'not use_MASTER - df is shuffled and reduced to {LEN}')
    print(f'k size: {len(df)}')

    config.j = config.j + '_reduced'

not use_MASTER - df is shuffled and reduced to 1000
k size: 1000


In [9]:




# try:#batch's stroma data if exists
#     stroma = config.pickle_load('k'+config.j[:-1]+'s'+'_dict', config.dir_data)['k'].reset_index(drop=True)
# except:
#     pass

# CAll = pd.concat([k.copy()])
print(f'{config.j} with size: {len(df)}')


# verify if needed; #remove them from the names_dict; they need to be only in cellcycle
names_dict = names_dict if '.2a' not in config.j else removeFeatures(names_dict.copy(),remove_features = ['CyclinB1', 'pRB','H3S28p','DNA2'])#

# NamesAll = names_dict['NamesAll']
# Core = names_dict['Core']

# ---------------------------------------
# groups = [['CellIden',              'EpiCols',              'CellIden+EpiCols', ],
#         [names_dict['CellIden'], names_dict['EpiCols'], names_dict['CellIden']+names_dict['EpiCols'],]]#EpiCols = ['EpiCols'] +['MBD']
groups = {'CellIden':names_dict['CellIden'],             'EpiCols':names_dict['EpiCols'] ,            'CellIden+EpiCols' :names_dict['CellIden']+names_dict['EpiCols'] ,
           'CellCycle': ['CyclinB1', 'pRB','H3S28p','DNA2'],
           'setA':['K5', 'GATA3', 'ER', 'H3K4me3', 'H3K27me3', 'H3K4me1', 'H3K9me2', 'H3K36me3', 'H4K20me3', 'H3K27me2', 'H3K9me3'],
          'Core' : names_dict['Core'],              'NamesAll': names_dict['NamesAll'],
            
            
          }


config.getNames(groups.keys())
config.features =  groups[config.name]
config.groups = groups; del groups
config.print()






b2345a_reduced with size: 1000
Sample is: batch
title; sampb2345a_reduced setA Based , figname; sampb2345a_reduced_setA_Based_


In [10]:
# p=0.99


# def predict_with_threshold(data,model ,label_encoder = None,thresh = 0.99,):
    
#     proba,preds  = model.predict_proba(data), model.predict(data)
#     preds[proba[np.arange(len(proba)),preds]<thresh] = len(np.unique(preds))#set all values that are not classified with high thresh to new class not in the original classes
#     if label_encoder is None:
#         return preds#don't inverse transform labels
#     labels = label_encoder.classes_
#     label_encoder.classes_ = np.append(labels, 'unclassified')#add the new class to the label encoder
#     preds =  label_encoder.inverse_transform(preds)
#     label_encoder.classes_ = labels#return the original classes to the label encoder
#     return preds
# ind_to_predict = classes3[classes3=='Noise'].index#predict only Noise & unknown
# for thresh in [0,0.1,0.2,0.2,0.5,0.7,0.9,0.99]:
#     preds = predict_with_threshold(data = df[config['features']].loc[ind_to_predict],thresh = thresh,model = model,label_encoder = label_encoder,)
#     print(thresh,len(preds[preds=='unclassified'])/len(preds),len(preds[preds=='unclassified']))
    


In [11]:


try:
    del names_dict,args,j
    config = config.get_dict()
except:
    pass
print(config.keys())
if not use_MASTER:
    # import pickle
    # with open(config['save_add']+'default_data'+'.p', "wb") as f:
    #     pickle.dump([df,config], file, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f'{config["save_add"]}default_sample.json', "w") as f:
        json.dump(df.to_dict(),f, )#indent = 4
    # print(config["dir_data"])
    # config["dir_data"] = config["save_add"]#change default saving and loading folder
    # print(config["dir_data"])
    with open(f'{config["save_add"]}default_config.json', "w") as f:
        json.dump(config,f, )#indent = 4 
    print(f'saved into: {config["save_add"]}')
    print (config['dir_plots'])
    print (config['dir_data'])
    del f


dict_keys(['j', 'feautures_ind', 'data_folder', 'visualize', 'plotUMAP', 'show', 'saveSVG', 'recalculate_umap', 'recalculate_db', 'create_adjusted', 'compare', 'isSingleSample', 'isCellCycle', 'home', 'dir_functions', 'dir_data_parent', 'dir_data', 'dir_index', 'dir_plots', 'save_add', 'name', 'title', 'figname', 'features', 'groups'])
saved into: /Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/
/Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/plots/_sample_b2345a_05062024_Data_/
/Users/yishai/Desktop/breast_cancer_PHD_research/phd_extras/data_yishai/Data_/
